In [71]:
import torch
import glob
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
from torch.optim import Adam

In [42]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [50]:
train_path='C:/Users/My LeNoVo/Jupyter Notebook Codes/seg_train/seg_train'
test_path='C:/Users/My LeNoVo/Jupyter Notebook Codes/seg_test/seg_test'

In [51]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

In [53]:
train_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),batch_size=256,shuffle=True)
test_loader=DataLoader(torchvision.datasets.ImageFolder(test_path,transform=transformer),batch_size=256,shuffle=False)

In [65]:
torch.cuda.is_available()

False

In [74]:
classes=['buildings','forest','glacier','mountain','sea','street']

In [69]:
#CNN Network
class ConvNet(nn.Module):
    def __init__(self):
        
        super(ConvNet,self).__init__()
        
        #Shape=(256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape=(256,12,150,150)
        
        self.bn1 = nn.BatchNorm2d(num_features=12)
        #Shape=(256,12,150,150)
        
        self.relu1=nn.ReLU()
        #Shape=(256,12,150,150)
        
        self.pool1=nn.MaxPool2d(kernel_size=2)
        #Shape=(256,12,75,75)
        
        
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,stride=1,padding=1)
        #Shape=(256,24,75,75)
        
        self.bn2=nn.BatchNorm2d(num_features=24)
        #Shape=(256,24,75,75)
        
        self.relu2=nn.ReLU()
        #Shape=(256,24,75,75)
        
        
        
        
        self.conv3=nn.Conv2d(in_channels=24,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape=(256,32,75,75)
        
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape=(256,32,75,150)
        
        self.relu3=nn.ReLU()
        #Shape=(256,32,75,75)
        
        self.pool3=nn.MaxPool2d(kernel_size=3)
        #Shape=(256,32,25,25)
        
        
        
        self.fc=nn.Linear(in_features=32*25*25, out_features=6)
        
    
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool1(output)
        
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
        
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=self.pool3(output)
        
        output=output.view(-1,32*25*25)
        output=self.fc(output)
        
        return output
        

In [70]:
model=ConvNet().to(device)

In [73]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_func=nn.CrossEntropyLoss()

In [77]:
train_count=len(glob.glob('C:/Users/My LeNoVo/Jupyter Notebook Codes/seg_train/seg_train/**/*.jpg'))
test_count=len(glob.glob('C:/Users/My LeNoVo/Jupyter Notebook Codes/seg_test/seg_test/**/*.jpg'))

In [78]:
best_accuracy=0.0
for epoch in range(2):
    
    train_accuracy=0.0
    test_accuracy=0.0
    
    model.train() 
    
    for i,(image,label) in enumerate(train_loader):
        image=image.to(device)
        label=label.to(device)
        
        optimizer.zero_grad()
        
        output=model(image)
        loss=loss_func(output,label)
        loss.backward()
        optimizer.step()
        
        _,prediction=torch.max(output.data,1)
        
        train_accuracy+=int(torch.sum(prediction==label.data))
    
    train_accuracy=(train_accuracy/train_count)
    
    model.eval()
    
    for i,(image,label) in enumerate(test_loader):
        image=image.to(device)
        label=label.to(device)
        
        optimizer.zero_grad()
        
        output=model(image)
        
        _,prediction=torch.max(output.data,1)
        
        test_accuracy+=int(torch.sum(prediction==label.data))
    
    test_accuracy=(test_accuracy/test_count)
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
    print(f'Epoch  {epoch+1}:   Train_accuracy:   {train_accuracy}     Test_accuracy:   {test_accuracy}'); 

Epoch  1:   Train_accuracy:   0.49123557075673363     Test_accuracy:   0.5896666666666667
Epoch  2:   Train_accuracy:   0.639162035057717     Test_accuracy:   0.6506666666666666
